<a href="https://colab.research.google.com/github/YuminosukeSato/CNN_research/blob/main/mnistCNN%2B%2Btest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import math
from sklearn.manifold import TSNE
from matplotlib import pyplot
import pickle
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
#import tqdm
from torch.autograd import Variable
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!git clone https://github.com/pyRiemann/pyRiemann

Cloning into 'pyRiemann'...
remote: Enumerating objects: 2529, done.
remote: Counting objects: 100% (814/814), done.
remote: Compressing objects: 100% (298/298), done.
remote: Total 2529 (delta 687), reused 535 (delta 515), pack-reused 1715
Receiving objects: 100% (2529/2529), 840.33 KiB | 6.89 MiB/s, done.
Resolving deltas: 100% (1727/1727), done.


In [14]:
class Net(nn.Module):

    def __init__(self):
        
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 60,kernel_size=(1,15),stride=(1,3))
        self.conv2 = nn.Conv2d(60, 60,kernel_size=(1,4),stride=(1,2))
        self.conv3 = nn.Conv2d(60, 60,kernel_size=(30,1),stride=(1,3))
        self.conv4 = nn.Conv2d(60, 90,kernel_size=(1,3),stride=(1,1),padding='same')
        self.conv5 = nn.Conv2d(90,120,kernel_size=(1,3),stride=(1,1),padding='same')
        self.pool = nn.MaxPool2d(kernel_size=(1,2),stride = (1,2))
        self.drop = nn.Dropout(p=0.5)
        self.bath1 = nn.BatchNorm2d(60)
        self.bath2 = nn.BatchNorm2d(90)
        self.bath3 = nn.BatchNorm2d(120)
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(120, 95)
        self.fc2 = nn.Linear(95, 10)
        self.flatten = nn.Flatten()
        self.drop = nn.Dropout(0.5)
        
    def forward(self, x):
      x = self.conv1(x)
      x = self.elu(self.bath1(self.drop(x)))
      x = self.pool(x)
      x = self.conv2(x)
      x = self.elu(self.bath1(self.drop(x)))
      x = self.pool(x)
      x = self.conv3(x)
      x = self.elu(self.bath1(self.drop(x)))
      x = self.conv4(x)
      x = self.elu(self.bath2(self.drop(x)))
      x = self.pool(x)
      x = self.conv5(x)
      x = self.elu(self.bath3(self.drop(x)))
      x = self.pool(x)
      x = self.flatten(x)
      x = self.fc1(x)
      x = self.drop(x)
      x = self.fc2(x)
      x = self.drop(x)
      #x = self.soft(x)
      return x

In [ ]:
cla

In [18]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
transform = transforms.Compose(
    [transforms.Resize([30,375]),
     transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))])
trainset = torchvision.datasets.MNIST(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                            batch_size=1000,
                                            shuffle=True,
                                            num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', 
                                        train=False, 
                                        download=True, 
                                        transform=transform)
testloader = torch.utils.data.DataLoader(testset, 
                                            batch_size=1000,
                                            shuffle=False, 
                                            num_workers=2)

classes = tuple(np.linspace(0, 9, 10, dtype=np.uint8))


In [ ]:
net = Net().to("cuda")
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(100):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        # zero the parameter gradients
        #print(inputs.shape)
        inputs = inputs.to("cuda")
        labels = labels.to("cuda")
        #print(inputs.shape)
        optimizer.zero_grad()
        # forward + backward + optimiz
        outputs = net(inputs).to("cuda")
        loss = criterion(outputs, labels).to("cuda")
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 60 == 59:
            print('[{:d}, {:5d}] loss: {:.3f}'
                    .format(epoch + 1, i + 1, running_loss / 60))
            running_loss = 0.0

print('Finished Training')

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fda2f3998c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fda2f3998c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
  File "/usr/lib/pytho

[1,    60] loss: 2.440


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fda2f3998c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fda2f3998c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

[2,    60] loss: 2.369


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fda2f3998c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fda2f3998c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

[3,    60] loss: 2.304


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for (images, labels) in testloader:
        p1d = (1, 346,2,0)
        images = F.pad(images, p1d, "constant", 1)
        outputs = net(images.to("cuda")).to("cuda")
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to("cuda")).sum().item()
print('Accuracy: {:.2f} %%'.format(100 * float(correct/total)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy: 76.31 %%


In [ ]:
def cal_acc(t,p):
    p_arg = torch.argmax(p,dim=1)
    return torch.sum(t == p_arg)

In [ ]:
test_total_acc = 0
net.eval()
#net_path = 'model.pth'
#net.load_state_dict(torch.load(net_path))
pred_list = []
true_list = []
with torch.no_grad():
    for n,(data,label) in enumerate(testloader):
        data = data.to("cuda")
        label = label.to("cuda")
        p1d = (1, 346,2,0)
        data = F.pad(data, p1d, "constant", 1)
        output = net(data).to("cuda")
        test_total_acc += cal_acc(label,output)
        pred = torch.argmax(output , dim =1)
        pred_list += pred.detach().cpu().numpy().tolist()
        true_list += label.detach().cpu().numpy().tolist() 
print(f"test acc:{test_total_acc/len(testset)*100}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


test acc:76.30999755859375


In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
def add_label(matrix,columns):
    columns_num = len(columns)
    act = ['正解データ'] * columns_num
    pred = ['予測データ'] * columns_num
    
    cm = pd.DataFrame(matrix,columns = [pred,columns],index = [act,columns])
    return cm
cm = add_label(confusion_matrix(true_list, pred_list),[x for x in range(10)])
display(cm)

予測データ                                             
            0    1    2    3    4    5    6    7    8    9
正解データ 0   971    0    0    3    0    2    0    0    2    2
      1     0  638    0  493    0    1    1    0    2    0
      2    21    7  672  268    8    2    0   33   15    6
      3     0    0    1  998    0    0    0    3    1    7
      4     9    0    0    4  684    1    0    0    2  282
      5    11    2    0  158    0  705    0    0    6   10
      6   220    4    1    7   18  101  471    7   76   53
      7     4    0    0  143    0    1    0  772    0  108
      8    12    0    0  118    1   14    0    3  751   75
      9     8    0    0   28    2    1    0    1    0  969